In [64]:
import pandas as pd
import pathlib
import numpy as np

## Load Data

In [115]:
raw_data_dir = pathlib.Path("../../data/raw/ms_work/")

# trans_res_path = raw_data_dir.joinpath('DSD_VLL_combined.csv')
trans_res_path = raw_data_dir.joinpath('Peggy_DSD_VLL_combined_remove_duplicates.csv')
trans_res_df_raw = pd.read_csv(trans_res_path)
trans_res_df_raw.columns = trans_res_df_raw.columns.map(str.lower)
trans_res_df_raw.head()

,peptide,replicate,precursor mz,precursor charge,product mz,product charge,fragment ion,area,background
0,DSDSGLLPR,D1_MCX_2_con_2ul_24,480.243263,2,480.243263,2,precursor,18077514,3605318
1,DSDSGLLPR,D2_MCX_2_con_2ul_23,480.243263,2,480.243263,2,precursor,21108704,1354337
2,DSDSGLLPR,D3_MCX_2_con_2ul_22,480.243263,2,480.243263,2,precursor,36066956,7820149
3,DSDSGLLPR,D4_MCX_2_con_2ul_21,480.243263,2,480.243263,2,precursor,16324817,1623649
4,DSDSGLLPR,D5_MCX_2_con_2ul_20,480.243263,2,480.243263,2,precursor,7801272,838242


In [117]:

trans_res_df_raw.peptide.unique()

array(['DSDSGLLPR', 'VLLEYADK', 'YQGFIFDIVTK', 'YFFSPTLFR', 'ALVGPGSPPR',
       'SAPGQLPSPR', 'AHSLNPDASGSSCSLAR', 'ASSADDIEAMR', 'SGLLNSTSDSDLVR'],
      dtype=object)

## Compute Area ratio

In [123]:
temp= trans_res_df_raw.copy()
temp['replicate'] = temp['replicate'].apply(fix_dilution)
cols_to_group_by = ['peptide',  'replicate', 'product charge', 'fragment ion']

temp = temp.groupby(cols_to_group_by).filter(lambda g: g['area'].count() in (1,3,7))
# temp[(temp['peptide'] == 'AHSLNPDASGSSCSLAR') &
# (temp['replicate'] == 'Peggy_SCN5A_D0_MCX_2_con_2ul_19') &
# (temp['product charge']=='2') &
# ('fragment ion'=='precursor')]
temp.sort_values(by=cols_to_group_by).to_csv(f'groups_odd_cnt_elements.csv')
temp.shape

(28, 9)

In [ ]:
# TODO: I need to first group, and then iterate through the groups and 
# then create a new colum that would make the group groupabe into twos
# e.g if 4 items in a group, for each items find difference closeds to 4 (chcek the peptide to know what value to use)
    # done by taking one item and iterating through others, soter the diffs in a list and check item loc closes to 4
    # that item loc isthe index of the pair area 
    # in the new col, the two pairs shouwl have same label 
    # do for the other items too
    # for group of foru, you end with 2 pairs



In [100]:


def grp_area_ratio_n_area_cols(grouped_df:pd.DataFrame):
    
    """compute the ratio of areas with higer precursor mz value as numerator."""

    df = grouped_df[['precursor mz', 'area']].copy()
    df = df.sort_values(by='precursor mz', ascending=True)
    area_ratio = df['area'].iloc[1] / df['area'].iloc[0]
    area_min, area_max  =  df['area'].iloc[0], df['area'].iloc[1]

    col_vals = {'area_ratio': area_ratio, 'area_min':area_min, 'area_max':area_max}

    return pd.Series(col_vals)

def fix_dilution(row:str):
    """fix dilution numbering"""
    str_to_check = row.split('_')[0]
    if len(str_to_check ) == 1:
        dilution = 'D'+row
    else:
        dilution = row

    return dilution

temp_df = trans_res_df_raw.copy()
temp_df['replicate'] = trans_res_df_raw['replicate'].apply(fix_dilution)

# def filter_group(grouped_df):
#     df = grouped_df.count(['area'])

cols_to_group_by = ['peptide',  'replicate', 'product charge', 'fragment ion']
print(temp_df.groupby(cols_to_group_by)['area'].count().value_counts())
# filter groups that do not contain exactlly teo element
temp_df = temp_df.groupby(cols_to_group_by).filter(lambda g: g['area'].count() == 2)
df_area_ratio = temp_df.groupby(cols_to_group_by).apply(grp_area_ratio_n_area_cols).reset_index()


# df_area_ratio = df_area_ratio[df_area_ratio['fragment ion'] == 'precursor']  # select only precursor
print(df_area_ratio.shape)
df_area_ratio.head()

2    658
4     34
1     12
8      9
3      3
7      1
Name: area, dtype: int64


c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in longlong_scalars
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


(658, 7)


,peptide,replicate,product charge,fragment ion,area_ratio,area_min,area_max
0,AHSLNPDASGSSCSLAR,D1_MCX_2_con_2ul_41,2,precursor,30.525728,377241.0,11515556.0
1,AHSLNPDASGSSCSLAR,D1_MCX_2_con_2ul_41,2,precursor [M+1],inf,0.0,7210099.0
2,AHSLNPDASGSSCSLAR,D1_MCX_2_con_2ul_41,2,precursor [M+2],0.646270,2670337.0,1725759.0
3,AHSLNPDASGSSCSLAR,D1_MCX_2_con_2ul_48,2,precursor,0.805933,4006649.0,3229092.0
4,AHSLNPDASGSSCSLAR,D1_MCX_2_con_2ul_48,2,precursor [M+1],4.073597,1950816.0,7946838.0


In [101]:
df_area_ratio.peptide.unique()

array(['AHSLNPDASGSSCSLAR', 'ALVGPGSPPR', 'DSDSGLLPR', 'SAPGQLPSPR',
       'SGLLNSTSDSDLVR', 'VLLEYADK', 'YFFSPTLFR', 'YQGFIFDIVTK'],
      dtype=object)

In [102]:
# df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('Peggy_conc.csv'))
# # df_peptide_dilution_conc.columns
# df_peptide_dilution_conc
# df_peptide_dilution_conc = df_peptide_dilution_conc.melt(id_vars='spiked concentration (ng/mL)', value_vars=['D1', 'D2', 'D3', 'D4', 'D5'], var_name='dilution')
# df_peptide_dilution_conc['peptide_dilution_name'] = df_peptide_dilution_conc['spiked concentration (ng/mL)'] + '_' + \
# df_peptide_dilution_conc['dilution'] 

## Get peptide Dilution Conc.

In [105]:

# TODO: should dilution name be fixed before grouping?
def get_dilution_from_replicate(rep_string:str):
    """Get the dilution symbol from the replicate name."""

    str_to_check = rep_string.split('_')[0]
    if str_to_check.startswith('col'):
        dilution = 'D0'
    elif len(str_to_check ) == 1:
        dilution = 'D'+str_to_check
    else:
        dilution = str_to_check

    return dilution

def get_peptide_dilution_comb(row):
    """function to create a new column from peptide and dilution name."""
    peptide = row['peptide']
    replicate  = row['replicate']
    dilution = get_dilution_from_replicate(rep_string=replicate)

    return peptide + '_' + dilution


df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('Peggy_conc.csv'))
# df_peptide_dilution_conc.columns
df_peptide_dilution_conc
df_peptide_dilution_conc = df_peptide_dilution_conc.melt(id_vars='spiked concentration (ng/mL)', 
                                                         value_vars=['D1', 'D2', 'D3', 'D4', 'D5'], 
                                                         var_name='dilution', value_name='heavy_conc')
peptide_dilution_name_col = 'peptide_dilution_name'
df_peptide_dilution_conc['peptide_dilution_name'] = df_peptide_dilution_conc['spiked concentration (ng/mL)'] + '_' + \
df_peptide_dilution_conc['dilution'] 

In [106]:
df_peptide_dilution_conc

,spiked concentration (ng/mL),dilution,heavy_conc,peptide_dilution_name
0,VLLEYADK,D1,49.400000,VLLEYADK_D1
1,YQGFIFDIVTK,D1,69.100000,YQGFIFDIVTK_D1
2,YFFSPTLFR,D1,61.200000,YFFSPTLFR_D1
3,HASFLFR,D1,45.600000,HASFLFR_D1
4,DSDSGLLPR,D1,49.800000,DSDSGLLPR_D1
5,VLLEYADK,D2,12.350000,VLLEYADK_D2
6,YQGFIFDIVTK,D2,17.275000,YQGFIFDIVTK_D2
7,YFFSPTLFR,D2,15.300000,YFFSPTLFR_D2
8,HASFLFR,D2,11.400000,HASFLFR_D2
9,DSDSGLLPR,D2,12.450000,DSDSGLLPR_D2


### Merge data

In [107]:
df_area_ratio[peptide_dilution_name_col] = df_area_ratio.apply(get_peptide_dilution_comb, axis=1)



df_area_ratio_conc = pd.merge(df_area_ratio, right=df_peptide_dilution_conc, how='left', 
                              on=peptide_dilution_name_col, suffixes=('', '_y'))
df_area_ratio_conc.drop(df_area_ratio_conc.filter(regex='_y$').columns, axis=1, inplace=True)
df_area_ratio_conc = df_area_ratio_conc.dropna(axis=0) # drop nan
# drop inf
df_area_ratio_conc.replace([np.inf, -np.inf], np.nan, inplace=True)
df_area_ratio_conc.dropna(subset=["area_ratio"], how="all", inplace=True)


def get_rep_plot_cat(rep_val:str):
    """Get the plot category (C18_SDS, C18_SDC, etc..)."""

    select_elements = rep_val.split('_')[1:3]

    cat = '_'.join(select_elements)
    return cat

def get_rep_plot_cat(row):
    """Get the plot category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']
    fragment_ion = row['fragment ion']

    rep_elements = replicate.split('_')[1:3]

    cat_name = [peptide] + [fragment_ion] + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

# create plot categories
df_area_ratio_conc['plot_cat'] = df_area_ratio_conc.apply(get_rep_plot_cat, axis=1)
df_area_ratio_conc.head()
# print(df_area_ratio_conc.isna().sum())

,peptide,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,spiked concentration (ng/mL),dilution,heavy_conc,plot_cat
66,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y1,9.535849,73517.0,701047.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.8,DSDSGLLPR_y1_MCX_2
67,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y2,399.510462,20599.0,8229516.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.8,DSDSGLLPR_y2_MCX_2
69,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y6,1378.872012,7782.0,10730382.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.8,DSDSGLLPR_y6_MCX_2
70,DSDSGLLPR,D1_MCX_2_con_2ul_24,2,precursor,28.006789,18077514.0,506293120.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.8,DSDSGLLPR_precursor_MCX_2
71,DSDSGLLPR,D1_MCX_2_con_2ul_24,2,precursor [M+1],31.552072,6195998.0,195496576.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.8,DSDSGLLPR_precursor [M+1]_MCX_2


## Get Linear Fit parameters

In [109]:
from sklearn import linear_model, metrics
import matplotlib.pyplot as plt

def get_linear_fit(data:pd.DataFrame, plot_cat:str):

    cat_data = data[data['plot_cat'] == plot_cat]
    # print(cat_data)

    # fig, ax = plt.subplots()

    x = cat_data['area_ratio'].values
    y = cat_data['heavy_conc'].values

    # ax.scatter(x, y)

    x = x[:, np.newaxis]
    y = y[:, np.newaxis]

    model = linear_model.LinearRegression()

    model.fit(x, y)

    y_fit = model.predict(x)

    # ax.plot(x.flatten(), y_fit.flatten())

    r2 = metrics.r2_score(y, y_fit)
    intercept = model.intercept_.squeeze()
    grad = model.coef_[0].squeeze()


    # print(f"R^2: {r2}")
    # print(f"intercept: {intercept}")
    # print(f"gradient:{grad}")

    # plt.show()
    
    return (r2, intercept, grad)

plot_cats = df_area_ratio_conc['plot_cat'].unique()

# plot_df = df_area_ratio_conc[df_area_ratio_conc.peptide=='GGLEPINFQTAADQAR'] # get only one peptide
# plot_df = plot_df[[peptide_dilution_name_col, 'area_ratio', 'heavy_conc', 'plot_cat']]
plot_df = df_area_ratio_conc


cats = []
r2s = []
intercepts = []
grads = []

for cat in plot_cats:
    # print(cat)
    r2, intercept, grad = get_linear_fit(plot_df, cat)
    cats.append(cat)
    r2s.append(r2)
    intercepts.append(intercept)
    grads.append(grad)
    



df_plot_cat_fit_params = pd.DataFrame({'plot_cat':cats,
              'R2':r2s,
              'intercept':intercepts,
              'gradient':grads})




c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [110]:
df_area_ratio_conc_n_fit_params = pd.merge(df_area_ratio_conc,
                                           right=df_plot_cat_fit_params,
                                           on='plot_cat',
                                           how='left')

df_area_ratio_conc_n_fit_params
# save to csv file
# df_area_ratio_conc_n_fit_params.to_csv('area_ratio_conc_n_fit_params.csv', index=False)
    

,peptide,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,spiked concentration (ng/mL),dilution,heavy_conc,plot_cat,R2,intercept,gradient
0,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y1,9.535849,73517.0,701047.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.800000,DSDSGLLPR_y1_MCX_2,0.676666,3.3679883507978374,5.065432757633233
1,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y2,399.510462,20599.0,8229516.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.800000,DSDSGLLPR_y2_MCX_2,0.804160,-0.3732158587012613,0.13395777305114376
2,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y6,1378.872012,7782.0,10730382.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.800000,DSDSGLLPR_y6_MCX_2,0.612880,5.21834983411043,0.04244755801799276
3,DSDSGLLPR,D1_MCX_2_con_2ul_24,2,precursor,28.006789,18077514.0,506293120.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.800000,DSDSGLLPR_precursor_MCX_2,0.555672,5.7767833731911695,1.1503898410493207
4,DSDSGLLPR,D1_MCX_2_con_2ul_24,2,precursor [M+1],31.552072,6195998.0,195496576.0,DSDSGLLPR_D1,DSDSGLLPR,D1,49.800000,DSDSGLLPR_precursor [M+1]_MCX_2,0.000846,14.76790365679255,-0.0013136201091829862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,YQGFIFDIVTK,D5_MCX_2_con_2ul_37,1,y1,0.000000,662.0,0.0,YQGFIFDIVTK_D5,YQGFIFDIVTK,D5,0.299913,YQGFIFDIVTK_y1_MCX_2,0.362550,22.748415750000003,2.7845622141771846
309,YQGFIFDIVTK,D5_MCX_2_con_2ul_37,2,precursor,0.083451,304922.0,25446.0,YQGFIFDIVTK_D5,YQGFIFDIVTK,D5,0.299913,YQGFIFDIVTK_precursor_MCX_2,0.043956,23.45448317091577,-0.3840406591498704
310,YQGFIFDIVTK,D5_MCX_2_con_2ul_37,2,precursor [M+1],89.747553,13385.0,1201271.0,YQGFIFDIVTK_D5,YQGFIFDIVTK,D5,0.299913,YQGFIFDIVTK_precursor [M+1]_MCX_2,0.029401,19.84482489027201,-0.23178676875370216
311,YQGFIFDIVTK,D5_MCX_2_con_2ul_37,2,precursor [M+2],0.157816,423233.0,66793.0,YQGFIFDIVTK_D5,YQGFIFDIVTK,D5,0.299913,YQGFIFDIVTK_precursor [M+2]_MCX_2,0.051021,21.213345876512157,-3.298724762535361


# Fit metric

In [2]:
q90 = [0.941, 0.765, 0.642, 0.56, 0.507, 0.468, 0.437,
       0.412, 0.392, 0.376, 0.361, 0.349, 0.338, 0.329,
       0.32, 0.313, 0.306, 0.3, 0.295, 0.29, 0.285, 0.281,
       0.277, 0.273, 0.269, 0.266, 0.263, 0.26
      ]

q95 = [0.97, 0.829, 0.71, 0.625, 0.568, 0.526, 0.493, 0.466,
       0.444, 0.426, 0.41, 0.396, 0.384, 0.374, 0.365, 0.356,
       0.349, 0.342, 0.337, 0.331, 0.326, 0.321, 0.317, 0.312,
       0.308, 0.305, 0.301, 0.29
      ]

q99 = [0.994, 0.926, 0.821, 0.74, 0.68, 0.634, 0.598, 0.568,
       0.542, 0.522, 0.503, 0.488, 0.475, 0.463, 0.452, 0.442,
       0.433, 0.425, 0.418, 0.411, 0.404, 0.399, 0.393, 0.388,
       0.384, 0.38, 0.376, 0.372
       ]

Q90 = {n:q for n,q in zip(range(3,len(q90)+1), q90)}
Q95 = {n:q for n,q in zip(range(3,len(q95)+1), q95)}
Q99 = {n:q for n,q in zip(range(3,len(q99)+1), q99)}

def dixon_test(data, left=True, right=True, q_dict=Q95):
    """
    Keyword arguments:
        data = A ordered or unordered list of data points (int or float).
        left = Q-test of minimum value in the ordered list if True.
        right = Q-test of maximum value in the ordered list if True.
        q_dict = A dictionary of Q-values for a given confidence level,
            where the dict. keys are sample sizes N, and the associated values
            are the corresponding critical Q values. E.g.,
            {3: 0.97, 4: 0.829, 5: 0.71, 6: 0.625, ...}

    Returns a list of 2 values for the outliers, or None.
    E.g.,
       for [1,1,1] -> [None, None]
       for [5,1,1] -> [None, 5]
       for [5,1,5] -> [1, None]

    """
    assert(left or right), 'At least one of the variables, `left` or `right`, must be True.'
    assert(len(data) >= 3), 'At least 3 data points are required'
    assert(len(data) <= max(q_dict.keys())), 'Sample size too large'

    sdata = sorted(data)
    Q_mindiff, Q_maxdiff = (0,0), (0,0)

    if left:
        Q_min = (sdata[1] - sdata[0])
        try:
            Q_min /= (sdata[-1] - sdata[0])
        except ZeroDivisionError:
            pass
        Q_mindiff = (Q_min - q_dict[len(data)], sdata[0])

    if right:
        Q_max = abs((sdata[-2] - sdata[-1]))
        try:
            Q_max /= abs((sdata[0] - sdata[-1]))
        except ZeroDivisionError:
            pass
        Q_maxdiff = (Q_max - q_dict[len(data)], sdata[-1])

    if not Q_mindiff[0] > 0 and not Q_maxdiff[0] > 0:
        outliers = [None, None]

    elif Q_mindiff[0] == Q_maxdiff[0]:
        outliers = [Q_mindiff[1], Q_maxdiff[1]]

    elif Q_mindiff[0] > Q_maxdiff[0]:
        outliers = [Q_mindiff[1], None]

    else:
        outliers = [None, Q_maxdiff[1]]

    return outliers


dixon_test([0.142, 0.153, 0.135, 0.002, 0.175], left=False)

[None, None]

In [111]:
def get_fit_param_cat(row):
    """Get the fit param category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']

    rep_elements = replicate.split('_')[1:3]

    cat_name = [peptide]  + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

def qtest(data, right=True):

    sorted_data = sorted(data)

    if right:
        gap = sorted_data[-1] - sorted_data[-2]

    else:
        gap = sorted_data[1] - sorted_data[0]

    try:
        range = sorted_data[-1] - sorted_data[0]
        q_val = gap / range
    except ZeroDivisionError:
        q_val = gap

    return q_val

def get_grp_fit_agg(grouped_df:pd.DataFrame):
    """Get the aggregate values for each group."""

    df = grouped_df[['intercept', 'gradient', 'R2']].copy()
    gradient = df['gradient'].values
    intercept = df['intercept'].values
    r2 = df['R2'].values

    mean_grad = gradient.mean()
    stdv_grad = gradient.std()
    cov_grad = stdv_grad / mean_grad
    qtest_grad_right = qtest(gradient, right=True)

    mean_intercept = intercept.mean()
    stdv_intercept = intercept.std()
    cov_intercept = stdv_intercept / mean_intercept
    qtest_intercept_right = qtest(intercept, right=True)

    mean_r2 = r2.mean()
    stdv_r2 = r2.std()
    cov_r2 = stdv_r2 / mean_r2
    qtest_r2_right = qtest(r2, right=True)

    cols_dict = {'mean_grad':mean_grad, 'stdv_grad':stdv_grad,
                 'cov_grad':cov_grad, 'qtest_grad':qtest_grad_right, 
                 'mean_intercept':mean_intercept, 'stdv_intercept':stdv_intercept,
                 'cov_intercept':cov_intercept, 'qtest_intercept':qtest_intercept_right,
                 'mean_r2':mean_r2, 'stdv_r2':stdv_r2,
                 'cov_r2':cov_r2, 'qtest_r2':qtest_r2_right
                 }

    return pd.Series(cols_dict)


# create plot categories
temp = df_area_ratio_conc_n_fit_params.copy()
temp['fit_param_grp'] = temp.apply(get_fit_param_cat, axis=1)

cols_to_group_by = ['fit_param_grp']
df_fit_param_agg = temp.groupby(cols_to_group_by).apply(get_grp_fit_agg).reset_index()

df_w_fit_param_agg = pd.merge(left=temp, right=df_fit_param_agg, how='left', on=cols_to_group_by)
df_w_fit_param_agg.head(3)

,peptide,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,spiked concentration (ng/mL),dilution,...,cov_grad,qtest_grad,mean_intercept,stdv_intercept,cov_intercept,qtest_intercept,mean_r2,stdv_r2,cov_r2,qtest_r2
0,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y1,9.535849,73517.0,701047.0,DSDSGLLPR_D1,DSDSGLLPR,D1,...,1.896481,0.0,7.141187,5.217154,0.730572,0.0,0.431049,0.326189,0.756733,0.0
1,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y2,399.510462,20599.0,8229516.0,DSDSGLLPR_D1,DSDSGLLPR,D1,...,1.896481,0.0,7.141187,5.217154,0.730572,0.0,0.431049,0.326189,0.756733,0.0
2,DSDSGLLPR,D1_MCX_2_con_2ul_24,1,y6,1378.872012,7782.0,10730382.0,DSDSGLLPR_D1,DSDSGLLPR,D1,...,1.896481,0.0,7.141187,5.217154,0.730572,0.0,0.431049,0.326189,0.756733,0.0


In [113]:
df_w_fit_param_agg.to_csv('peggy_data_with_fit_param_aggregate.csv', index=False)